In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary



C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
# Datas preprocessing methods.
# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw

def lemmatize(article):
    output = []
    outPonc = article.translate(article.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [4]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [5]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

# To create ours delimiters, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    if not isinstance(data['abstract'][i], float): #if not null, empty, or whatever (so if there is a abstract)
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
        numArti+=1
        usable.append(stemmer.stem(lemmatize(data['abstract'][i])))
limits.append(numArti)



In [6]:
# Display pre-processed datas
print("nombre d'articles =", len(usable))
print("limits =", limits)

usable[0]

nombre d'articles = 1096
limits = [267, 543, 790, 1096]


'plateforme objectif permettre citoyen danalyserpar euxmême tweet politique dévénement spécifique francepour cas lélection présidentiel 2017 idéo2017 analyser quasitemp réel message candidat fournir principal caractéristiqueslusage lexiqu politique comparaison entrer candidat'

In [7]:
#params
nb_concepts = 30
min_gram = 1
max_gram = 3

# Creation of cleandocs, which is usable[] with ngrams
cleandocs = []
for t in usable:
    doc = []
    for n in range(min_gram, max_gram+1):
        for gram in ngrams(t.split(), n):
            doc.append(" ".join(gram))
    cleandocs.append(doc)

# Creation of tfidf model, a tool to create ours tfidf
corpus = []
dictionary = corpora.Dictionary(cleandocs)
for doc in cleandocs:
    newVec = dictionary.doc2bow(doc)
    corpus.append(newVec)
tfidf = models.TfidfModel(corpus)

# Creation of partitions_lsa[], which give us the LSA of each partition
partitions_lsa = []
beg = 0
for l in limits:
    last = l
    corpus_tfidf = tfidf[corpus]
    lsi = models.LsiModel(corpus_tfidf, num_topics=nb_concepts, id2word=dictionary)
    corpus_lsi = lsi[corpus_tfidf[beg:last]]
    partitions_lsa.append(corpus_lsi)
    beg = l

In [8]:
num_partition = 0
for lsa in partitions_lsa:
    print("Partition numéro:",num_partition)
    num_partition+=1
    i=0
    for doc in lsa:
        if (i<3):
            print("document number ", i)
            i+=1
            print(doc)


Partition numéro: 0
document number  0
[(0, -0.00442525458643619), (1, -0.02960472232489413), (2, 0.007187271261485128), (3, -0.010163289400516612), (4, 0.027906869287719065), (5, 0.013444671077312129), (6, -0.01601118924958787), (7, 0.05147121427214083), (8, 0.033533235738763094), (9, 0.00368402146607114), (10, -0.019326838155898494), (11, 0.0076580913305048), (12, 0.004982137500364499), (13, -0.03665251238755576), (14, 0.0004213584272149639), (15, -0.021140695382502975), (16, -0.0015540745732613322), (17, -0.009233778724669387), (18, 0.02040349931749458), (19, 0.04948363822768231), (20, 0.019836501575363107), (21, -0.0008547899353163455), (22, 0.06769213815517623), (23, 0.0210695776207959), (24, -0.013398617928773749), (25, 0.004508298536446662), (26, -0.010083838320645534), (27, 0.03349603308043118), (28, 0.020394990040406753), (29, 0.004775044082856077)]
document number  1
[(0, -0.011020756955887043), (1, -0.09853747400540062), (2, -0.027366706972146194), (3, 0.07079287849658708), 

[(0, 0.00843419546398208), (1, -0.09146406420927741), (2, -5.182104837538671e-05), (3, 0.08922833102519247), (4, -0.09157097353803897), (5, 0.002680083143680682), (6, -0.03890003321671587), (7, 0.012141631761116803), (8, 0.000828875250456516), (9, 0.04545622766806511), (10, 0.028978874715015666), (11, 0.0481188583762124), (12, 0.059078413119139965), (13, 0.04457230887816985), (14, -0.008549044519596434), (15, -0.008787877070289921), (16, -0.006763338680368765), (17, 0.02459017835323135), (18, -0.015989719636662736), (19, -0.04670852952040745), (20, -0.09638697591361216), (21, -0.004135618210438457), (22, 0.045538371001288075), (23, -0.010395535437275383), (24, 0.02956146091692291), (25, -0.022389549076294302), (26, 0.011714135407949031), (27, -0.05987985916321611), (28, 0.04536238181688116), (29, -0.067337245849176)]
document number  1
[(0, 0.008745546771916043), (1, -0.07340575375494054), (2, 0.01928723496240723), (3, 0.023590707342729833), (4, 0.016565250482938054), (5, -0.0225176894

In [9]:
# Let's create ours partitions
partitions = []

# You must specify a treshold, to know what are the doc you keep, and what are the doc you drop
tresh = 0.03

for corpus_lsi in partitions_lsa:
    # Let's create ours clusters
    clusters = []

    for i in range(0, nb_concepts):
        dic = {}
        num_doc = 0
        for doc in corpus_lsi:
            if abs(doc[i][1]) > tresh:
                dic[num_doc] = doc[i][1]
            num_doc+=1
        clusters.append(dic)
    partitions.append(clusters)
    
# TODO: it would be nice to know how many articles are in no cluster anymore

In [10]:
# Display clusters 3 of partition 0 
partitions[0][3]

{1: 0.07079287849658708,
 3: 0.06278657173783952,
 4: -0.041618199069723756,
 8: 0.054975517638219194,
 10: 0.05221771109005783,
 13: -0.03192847468678608,
 14: -0.03791024301073565,
 15: -0.05955978337700095,
 19: -0.03489750315996548,
 21: -0.033238977202303434,
 22: -0.047209915155047084,
 27: -0.09349683388104621,
 30: -0.043076981097785585,
 34: 0.04103058699427752,
 39: 0.06067031860007079,
 43: -0.04072268230397381,
 44: -0.06695194373437295,
 45: 0.04050302608224564,
 50: 0.0710103730320674,
 51: -0.030669369773278516,
 54: -0.04483703035037884,
 55: -0.033769739510406106,
 56: -0.052527365658544496,
 59: 0.03935718679638772,
 60: 0.09441085781127445,
 63: -0.047920962467156866,
 66: 0.031567115707090664,
 69: 0.0914453877946898,
 78: -0.049187510489126986,
 83: -0.05612234523734906,
 85: -0.048745281608375345,
 86: 0.0436273860780593,
 92: -0.033202635275020355,
 93: 0.04855058050023327,
 94: 0.04032940601720302,
 97: -0.05275780041899206,
 99: 0.10614847786250386,
 100: 0.030

In [11]:
nb_labels_by_cluster = 5

# Let's labelize our clusters
# For this, we will use the tfidf matrix

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram))
tfidf = vectorizer.fit_transform(usable)

# We can access the value in the tfidf using:
#tfidf.toarray().item(num_doc, num_word)
# To know the number of the word searched, we will use:
#vectorizer.vocabulary_[word]

# take less than 8h to compute x)
labels = []
for clusters in partitions:
    l = []
    for clus in clusters:
        first_arti = True
        for article in clus:
            link = abs(clus[article])
            if first_arti:
                coef_list = (tfidf.toarray()[article] * link)
                first = False
            else:
                # the more an article have a high coeficient, the more he is implied in the labeling step
                coef_list += (tfidf.toarray()[article] * link)
        # Now we have coef_list filled by every coeficient in the multiple tfidf
        # Let's find the best ones, to finally get the labels
        res = dict(zip(vectorizer.get_feature_names(), coef_list))

        l.append(Counter(res).most_common(nb_labels_by_cluster))
    labels.append(l)

# TODO: on observe beaucoup de labels identiques entre deux clusters
# Je pense que c'est parce que l'on a trop de clusters, mais j'aimerais en être sûr

In [12]:
# Display labels
# labels is composed by an array for each partition
labels

[[[('xplor', 0.03970108413359015),
   ('xplor everywhere', 0.03970108413359015),
   ('everywhere', 0.03749812918607926),
   ('and', 0.028002371100805695),
   ('dater', 0.020009476569576027)],
  [('hotspot', 0.027371980004046396),
   ('relation', 0.006556304832916206),
   ('photographie', 0.006463287241834405),
   ('détecter', 0.003906684431161257),
   ('approcher extraire relation', 0.0034214975005057995)],
  [('lien', 0.006192576191291464),
   ('rankmerging', 0.005587713002789331),
   ('dan grand', 0.004527631369350621),
   ('réseau', 0.0039175314170584565),
   ('article décrire rankmerging', 0.0027938565013946653)],
  [('petl', 0.013622325033983564),
   ('donnée massif', 0.0059570712163714715),
   ('paramétrage', 0.005222135732499675),
   ('massif', 0.004712692377045261),
   ('parallèle', 0.004619402760206431)],
  [('réseau social', 0.013993670644504144),
   ('social', 0.01261600876588804),
   ('baser interaction', 0.01164783189524735),
   ('baser interaction utilisateur', 0.01164783

In [26]:
# Diachronic analysis

vectorizer = TfidfVectorizer(stop_words=fstw, use_idf=True, ngram_range=(min_gram, max_gram))
tfidf = vectorizer.fit_transform(usable)

# Feature Recall
# label_f: num of the label
def FR(cluster_c, label_f, partition_C):
    total_singledoc = 0
    for doc in cluster_c:
        total_singledoc += tfidf.toarray().item(doc, label_f)
    total_everydoc = 0
    for cluster in partition_C:
        for doc in cluster:
            total_everydoc += tfidf.toarray().item(doc, label_f)
    return total_singledoc / total_everydoc

# Feature Precision
def FP(cluster_c, label_f):
    total_singledoc = 0
    for doc in cluster_c:
        total_singledoc += tfidf.toarray().item(doc, label_f)
    total_everydoc = 0
    for doc in cluster_c:
        total_everydoc += sum(tfidf.toarray()[doc])
    return total_singledoc / total_everydoc
    
# Feature F-measure
def FF(cluster_c, label_f, partition_C):
    res = 2*FR(cluster_c, label_f, partition_C)*FP(cluster_c, label_f)
    res /= (FR(cluster_c, label_f, partition_C) + FP(cluster_c, label_f))
    return res

In [27]:
# Vamos a implementar una nueva tecnicà para labelisar: FF-measure

# list of labels of cluster C from partition P = labels[P][C]
labels = []


# first, we want the mean ffmeasure of each f
ffmeanF = []

for f in range(0, len(tfidf.toarray()[0])):
    mean = 0
    nb = 0
    for clusters in partitions:
        for clus in clusters:
            mean += FF(clus, f, clusters)
            nb += 1
    mean /= nb
    ffmeanF.append(mean)
    

#TODO: question: on prend on compte les zeros (genre quand le mot n'apparait pas)?
# second, we want the mean of every ffmeasure
ffmean_total

mean = 0
nb = 0
for f in range(0, len(tfidf.toarray()[0])):
    for clusters in partitions:
        for clus in clusters:
            mean += FF(clus, f, clusters)
            nb += 1
fflean_total = mean / nb

# Now we fill labels[]
for clusters in partitions:
    labels_for_clusters = []
    for clus in clusters:
        labels_for_clus = []
        for arti in clus:
            for num_word in range(0, len(tfidf.toarray()[arti])):
                if tfidf.toarray().item(arti, num_word) != 0:
                    ffmes = FF(clus, num_word, partitions)
                    if ffmes > ffmeanF[num_word] and ffmes > ffmean_total:
                        labels_for_clus.append(num_word)
        labels_for_clusters[clus] = labels_for_clus
    labels[clusters] = labels_for_clusters



MemoryError: 

In [ ]:
# TODO: sigma are ecart-type :)

def inter(listA, listB):
    return np.intersect1d(listA, listB)
    
# cluster_t and cluster_s must be in two different partitions
def proba(cluster_t, cluster_s, partition_T, partition_S):
    total_inter = 0
    for f in inter(labels[partition_T][cluster_t], labels[partition_S][cluster_s]):
        # Beware! I know I am passing the whole tfidf, but hang on, that's just to understand now
        total_inter += FF(cluster_t, f, partition_T)
    total_t = 0
    for f in labels[partition_T][cluster_t]:
        total_t += FF(cluster_t, f, partition_T)
    return total_inter / total_t
    

def P_A(cluster_s, partition_T, partition_S):
    # first, we have to know what are the cluster which got the label
    total = 0
    nb_computation = 0
    for label_s in labels[partition_S][num_cluster_s]:
        for cluster_t in partition_T:
            if label_s in labels[partition_T][cluster_t]:
                total += proba(cluster_t, cluster_s, partition_T, partition_S)
                nb_computation += 1
    return total / nb_computation

# Define a coeficient for the activity 
def activity(partition_S, partition_T):
    res = 0
    for cluster_s in partition_S:
        res += P_A(cluster_s, partition_T, partition_S)
    return res / len(partition_S)

# Standard deviation
# Nothing have been find in the paper, so I put those random values ¯\_(ツ)_/¯
sigma_t = 0.01
sigma_s = 0.01

# Our Graal
# Does cluster_t is similar to cluster_s?
def similar(cluster_t, cluster_s, partition_T, partition_S):
    cond1 = proba(cluster_t, cluster_s, partition_T, partition_S) > P_A(cluster_s, partition_T, partition_S)
    cond2 = proba(cluster_t, cluster_s, partition_T, partition_S) > activity(partition_S, partition_T) + sigma_s
    
    cond1 = proba(cluster_t, cluster_s, partition_T, partition_S) > P_A(cluster_s, partition_T, partition_S)
    cond2 = proba(cluster_t, cluster_s, partition_T, partition_S) > activity(partition_T, partition_S) + sigma_t
    return cond1 and cond2 and cond3 and cond4
    